In [28]:
# !pip install optuna
# !pip install tqdm
# !pip install feature-engine
# !pip install scikit-learn
# !pip install lightgbm
# !pip install --upgrade scikit-learn
# !pip install --upgrade <cyclical-features-library>


In [29]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import optuna
from sklearn.model_selection import RepeatedKFold
from tqdm import tqdm
import lightgbm as lgb
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_log_error
from sklearn.preprocessing import LabelEncoder
from datetime import datetime
from feature_engine.creation import CyclicalFeatures
import warnings
warnings.filterwarnings('ignore')

In [30]:
#Loading the Data

train = pd.read_csv("./train.csv")
test = pd.read_csv("./test.csv")

Data Preprocessing
-> Clean the data by adding the mean value to the fields where it is missing or a random value where the mean can t be calculated
-> dropping the id column

In [31]:

mean_age = train['Age'].mean()
train['Age'] = train['Age'].fillna(mean_age)
test['Age'] = test['Age'].fillna(mean_age)

median_annual_income = train['Annual Income'].median()
train['Annual Income'] = train['Annual Income'].fillna(median_annual_income)
test['Annual Income'] = test['Annual Income'].fillna(median_annual_income)

train = train.dropna(subset=['Marital Status'])
test['Marital Status'] = test['Marital Status'].apply(
    lambda x: np.random.choice(['Single', 'Married', 'Divorced'],
                               p=[0.3347, 0.3337, 0.3316]) if pd.isna(x) else x
)

mean_number_dependents = train['Number of Dependents'].mean()
train['Number of Dependents'] = train['Number of Dependents'].fillna(mean_number_dependents)
test['Number of Dependents'] = test['Number of Dependents'].fillna(mean_number_dependents)

train['Occupation'] = train['Occupation'].fillna('Missing')
test['Occupation'] = test['Occupation'].fillna('Missing')

mean_health_score = train['Health Score'].mean()
train['Health Score'] = train['Health Score'].fillna(mean_health_score)
test['Health Score'] = test['Health Score'].fillna(mean_health_score)

data_vc_train = train.dropna(subset=['Vehicle Age'])
mean_vehicle_age = test['Vehicle Age'].mean()
test['Vehicle Age'] = test['Vehicle Age'].fillna(mean_vehicle_age)

mean_credit_score = train['Credit Score'].mean()
train['Credit Score'] = train['Credit Score'].fillna(mean_credit_score)
test['Credit Score'] = test['Credit Score'].fillna(mean_credit_score)

mean_insurance_duration = train['Insurance Duration'].mean()
train['Insurance Duration'] = train['Insurance Duration'].fillna(mean_insurance_duration)
test['Insurance Duration'] = test['Insurance Duration'].fillna(mean_insurance_duration)

train['Customer Feedback'] = train['Customer Feedback'].fillna('Unknown')
test['Customer Feedback'] = test['Customer Feedback'].fillna('Unknown')

train.drop(columns=['id'], inplace=True)
test.drop(columns=['id'], inplace=True)

In [32]:
#Cycling dates
train['Policy Start Date'] = pd.to_datetime(train['Policy Start Date'])
test['Policy Start Date'] = pd.to_datetime(test['Policy Start Date'])

# Cyclical encoding of the year - no need for cos and sinus interpretations
train['Year'] = train['Policy Start Date'].dt.year
test['Year'] = test['Policy Start Date'].dt.year

train['Month'] = train['Policy Start Date'].dt.month
test['Month'] = test['Policy Start Date'].dt.month

train['Month_cos'] = np.cos(2 * np.pi * train['Month'] / 12)
train['Month_sin'] = np.sin(2 * np.pi * train['Month'] / 12)

test['Month_cos'] = np.cos(2 * np.pi * test['Month'] / 12)
test['Month_sin'] = np.sin(2 * np.pi * test['Month'] / 12)

train['Day'] = train['Policy Start Date'].dt.day
test['Day'] = test['Policy Start Date'].dt.day

train['Day_cos'] = np.cos(2 * np.pi * train['Day'] / 30)
train['Day_sin'] = np.sin(2 * np.pi * train['Day'] / 30)

test['Day_cos'] = np.cos(2 * np.pi * test['Day'] / 30)
test['Day_sin'] = np.sin(2 * np.pi * test['Day'] / 30)

train['Group'] = (train['Year'] - 2020) * 48 + train['Month'] * 4 + train['Day'] // 7
test['Group'] = (test['Year'] - 2020) * 48 + test['Month'] * 4 + test['Day'] // 7

train.drop(columns=['Day','Month','Policy Start Date'], inplace=True)
test.drop(columns=['Day','Month','Policy Start Date' ], inplace=True)

In [49]:
#encoding the categorical columns
le = LabelEncoder()

for col in train.columns:
    if train[col].dtype == 'object':
        train[col] = le.fit_transform(train[col])
        test[col] = le.transform(test[col])

In [34]:
# #Convert categorical columns to category dtype for Catboost
# categorical_columns = ['Gender', 'Marital Status', 'Education Level', 'Occupation', 
#                        'Location', 'Policy Type', 'Previous Claims', 'Smoking Status', 
#                        'Exercise Frequency', 'Property Type', 'Customer Feedback']

# for col in categorical_columns:
#     test[col] = test[col].astype('category')
#     train[col] = train[col].astype('category')


In [35]:
from sklearn.model_selection import train_test_split

y = train['Premium Amount']
X = train.drop(columns=['Premium Amount'])

X_train, X_ref, y_train, y_ref = train_test_split(X, y, test_size=0.2, random_state=42)


In [36]:
def rmsLe(y_true, y_pred):
    y_pred = np.maximum(y_pred, 1e-6)
    return np.sqrt(mean_squared_log_error(y_true, y_pred))

# SEED = 42
# n_splits = 5

# def TrainML(params, e_stop=50):
#     kfold = RepeatedKFold(n_splits=n_splits, n_repeats=1, random_state=SEED)
#     train_rmse_scores = []
#     val_rmse_scores = []

#     for fold, (train_idx, val_idx) in enumerate(tqdm(kfold.split(X, y), desc="Training Folds", total=n_splits)):
#         X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
#         y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

#         y_train_log = np.log1p(y_train)
#         y_val_log = np.log1p(y_val)
        
#         callbacks = [lgb.early_stopping(stopping_rounds=e_stop, verbose=False)]
#         model = lgb.LGBMRegressor(**params, random_state=SEED, verbose=-1, n_jobs=-1)
#         model.fit(X_train, y_train_log, 
#                   eval_set=[(X_val, y_val_log)], 
#                   eval_metric='rmse', 
#                   callbacks=callbacks)
        
#         y_train_log_pred = model.predict(X_train)
#         y_val_log_pred = model.predict(X_val)
        
#         y_train_pred = np.expm1(y_train_log_pred)
#         y_val_pred = np.expm1(y_val_log_pred)
        
#         train_rmse = rmsLe(y_train, y_train_pred)
#         val_rmse = rmsLe(y_val, y_val_pred)

#         train_rmse_scores.append(train_rmse)
#         val_rmse_scores.append(val_rmse)

#     mean_train_rmse = np.mean(train_rmse_scores)
#     mean_val_rmse = np.mean(val_rmse_scores)

#     print("\n Final Mean Scores:")
#     print(f" Mean Train RMSLE: {mean_train_rmse:.4f}")
#     print(f" Mean Validation RMSLE: {mean_val_rmse:.4f}")

#     return mean_train_rmse, mean_val_rmse

# def objective(trial):
#     params = {
#         'n_estimators': trial.suggest_int('n_estimators', 100, 1000, step=50),
#         'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.2, log=True),
#         'num_leaves': trial.suggest_int('num_leaves', 20, 150),
#         'max_depth': trial.suggest_int('max_depth', 3, 15),
#         'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
#         'subsample': trial.suggest_float('subsample', 0.5, 1.0),
#         'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
#         'reg_alpha': trial.suggest_float('reg_alpha', 1e-4, 10.0, log=True),
#         'reg_lambda': trial.suggest_float('reg_lambda', 1e-4, 10.0, log=True),
#         'device': 'gpu'
#     }

#     mean_train_rmse, mean_val_rmse = TrainML(params)
#     trial.set_user_attr('train_rmse', mean_train_rmse)
#     trial.set_user_attr('val_rmse', mean_val_rmse)
#     return mean_val_rmse

# study = optuna.create_study(direction='minimize')
# study.optimize(objective, n_trials=5)


In [37]:
# from catboost import CatBoostRegressor

# for col in categorical_columns:
#     # Ensure the column exists in both datasets
#     if col in X_train.columns and col in X_ref.columns:
#         if X_train[col].dtype.name == "category":
#             # Check if "missing" is already a category
#             if "missing" not in X_train[col].cat.categories:
#                 X_train[col] = X_train[col].cat.add_categories(["missing"])
#                 X_ref[col] = X_ref[col].cat.add_categories(["missing"])
        
#         # Handle missing values and convert to string
#         X_train[col] = X_train[col].fillna("missing").astype(str)
#         X_ref[col] = X_ref[col].fillna("missing").astype(str)

# # Process all categorical features to ensure consistency
# for col in categorical_columns:
#     X_train[col] = X_train[col].fillna("missing").astype(str)
#     X_ref[col] = X_ref[col].fillna("missing").astype(str)

# cat_features = categorical_columns

# best_params_catboost = {'iterations': 50, 'learning_rate': 0.006002641114390338, 'depth': 15, 'l2_leaf_reg': 0.40630468090624405,
#     'subsample': 0.7200344333898133, 'colsample_bylevel': 0.6986521753741868, 'min_data_in_leaf': 72, 'random_state': 42, 
#     'cat_features': categorical_columns, 'verbose': 200}

# catboost_model = CatBoostRegressor(**best_params_catboost)

# best_params = {'n_estimators': 700, 'learning_rate': 0.006002641114390338, 'num_leaves': 42, 'max_depth': 15, 'min_child_samples': 72, 
#                'subsample': 0.7200344333898133, 'colsample_bytree': 0.6986521753741868, 'reg_alpha': 0.0013223748424590466, 
#                'reg_lambda': 0.40630468090624405}

# #Training LGBMRegressor Model - It is known for being fast and efficient in training, especially on large datasets.
# LGBMR_model = LGBMRegressor(**best_params, verbose=-1, n_jobs=-1)
# LGBMR_model.fit(X_train, np.log1p(y_train))#this log is recommended when the distribution is skewed

# #Make predictions
# test_preds = LGBMR_model.predict(X_ref)
# test_preds = np.expm1(test_preds)

# error_lgbm = rmsLe(y_ref, test_preds)

# # sample_submission['Premium Amount'] = test_preds
# # sample_submission.to_csv('submission1.csv', index=False)

# #Visualising the importance of each field of the dataset in determing the Premium Amount
# importances = LGBMR_model.feature_importances_
# norm_importances = importances / np.linalg.norm(importances)
# features = X.columns
# importance_df = pd.DataFrame({'Feature': features, 'Importance': norm_importances})
# importance_df.sort_values(by='Importance', ascending=False, inplace=True)

# plt.figure(figsize=(12, 8))
# sns.barplot(x='Importance', y='Feature', data=importance_df)
# plt.title('Error = {:.5f}'.format(error_lgbm))
# plt.show()
# Error = 1.05377

In [38]:
# from xgboost import XGBRegressor

# best_params_xgb = {'n_estimators': 700, 'learning_rate': 0.006002641114390338, 'max_depth': 15, 'min_child_weight': 72,
#                    'subsample': 0.7200344333898133, 'colsample_bytree': 0.6986521753741868, 'reg_alpha': 0.0013223748424590466,
#                     'reg_lambda': 0.40630468090624405}

# xgb_model = XGBRegressor(**best_params_xgb, objective='reg:squarederror', random_state=42)
# xgb_model.fit(X_train, np.log1p(y_train))

# test_preds = xgb_model.predict(X_ref)
# test_preds = np.expm1(test_preds)

# error_xgb = rmsLe(y_ref, test_preds)

# # sample_submission['Premium Amount'] = test_preds
# # sample_submission.to_csv('submission1.csv', index=False)

# #Visualising the importance of each field of the dataset in determing the Premium Amount
# importances = xgb_model.feature_importances_
# norm_importances = importances / np.linalg.norm(importances)
# features = X.columns
# importance_df = pd.DataFrame({'Feature': features, 'Importance': norm_importances})
# importance_df.sort_values(by='Importance', ascending=False, inplace=True)

# plt.figure(figsize=(12, 8))
# sns.barplot(x='Importance', y='Feature', data=importance_df)
# plt.title('Error = {:.5f}'.format(error_xgb))
# plt.show()
# #Error = 1.05411

Catboost model

In [39]:
# !pip uninstall numpy catboost
# !pip install numpy catboost


In [50]:
# from catboost import CatBoostRegressor

# for col in categorical_columns:
#     # Ensure the column exists in both datasets
#     if col in X_train.columns and col in X_ref.columns:
#         if X_train[col].dtype.name == "category":
#             # Check if "missing" is already a category
#             if "missing" not in X_train[col].cat.categories:
#                 X_train[col] = X_train[col].cat.add_categories(["missing"])
#                 X_ref[col] = X_ref[col].cat.add_categories(["missing"])
        
#         # Handle missing values and convert to string
#         X_train[col] = X_train[col].fillna("missing").astype(str)
#         X_ref[col] = X_ref[col].fillna("missing").astype(str)

# # Process all categorical features to ensure consistency
# for col in categorical_columns:
#     X_train[col] = X_train[col].fillna("missing").astype(str)
#     X_ref[col] = X_ref[col].fillna("missing").astype(str)

# cat_features = categorical_columns

# best_params_catboost = {'iterations': 50, 'learning_rate': 0.006002641114390338, 'depth': 15, 'l2_leaf_reg': 0.40630468090624405,
#     'subsample': 0.7200344333898133, 'colsample_bylevel': 0.6986521753741868, 'min_data_in_leaf': 72, 'random_state': 42, 
#     'cat_features': categorical_columns, 'verbose': 200}

# catboost_model = CatBoostRegressor(**best_params_catboost)

# catboost_model.fit(X_train, np.log1p(y_train))

# test_preds = catboost_model.predict(X_ref)
# test_preds = np.expm1(test_preds)  # Transform predictions back from log space

# # Convert predictions to a DataFrame
# sample_submission = pd.DataFrame({'Premium Amount': test_preds})
# sample_submission.to_csv('submissionCATBOOST.csv', index=False)

# # error_catboost = rmsLe(y_ref, test_preds)

# # #Visualising the importance of each field of the dataset in determing the Premium Amount
# # importances = catboost_model.feature_importances_
# # norm_importances = importances / np.linalg.norm(importances)
# # features = X.columns
# # importance_df = pd.DataFrame({'Feature': features, 'Importance': norm_importances})
# # importance_df.sort_values(by='Importance', ascending=False, inplace=True)

# # plt.figure(figsize=(12, 8))
# # sns.barplot(x='Importance', y='Feature', data=importance_df)
# # plt.title('Error = {:.5f}'.format(error_catboost))
# # plt.show()
